In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

import joblib
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

In [141]:
apart = pd.read_excel(''+'train.xlsx')
apart

,단지코드,단지명,총세대수,전용면적별세대수,지역,준공일자,건물형태,난방방식,승강기설치여부,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수
0,C0001,엘에이치 서초4단지,78,35,서울,20131204,계단식,개별가스난방,전체동 설치,120,51.89,19.2603,50758000,620370,109
1,C0001,엘에이치 서초4단지,78,43,서울,20131204,계단식,개별가스난방,전체동 설치,120,59.93,22.2446,63166000,665490,109
2,C0002,LH삼성아파트,35,26,서울,20130801,복도식,개별가스난방,전체동 설치,47,27.75,16.5375,63062000,458640,35
3,C0002,LH삼성아파트,35,9,서울,20130801,복도식,개별가스난방,전체동 설치,47,29.08,17.3302,63062000,481560,35
4,C0003,강남LH8단지,88,7,서울,20131023,계단식,개별가스난방,전체동 설치,106,59.47,21.9462,72190000,586540,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,C0356,덕유마을 주공1단지,956,956,경기,19941130,복도식,지역가스난방,전체동 설치,202,26.37,12.7500,9931000,134540,243
1153,C0358,화천신읍(공공실버) 영구임대,120,66,강원,20200110,복도식,NaN,전체동 설치,40,24.83,15.1557,2129000,42350,47
1154,C0358,화천신읍(공공실버) 영구임대,120,54,강원,20200110,복도식,NaN,전체동 설치,40,33.84,20.6553,2902000,57730,47
1155,C0359,영천야사4,447,149,대구경북,19940615,복도식,중앙유류난방,전체동 설치,89,26.37,13.3800,7134000,118880,78


# 건물 형태에 대한 결측치를 KNN으로 예측을 해야함

- 또는 Kmeans나 어떤 것들로해서 예측을 해야함

In [143]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [145]:
drop_cols = ['단지명','단지코드','지역','난방방식','승강기설치여부'] # 범주형 칼럼은 가변수화에 있어 문제가 생겨 아예 삭제하는 방향으로 잡음
apart = apart.drop(columns=drop_cols)
apart

,총세대수,전용면적별세대수,준공일자,건물형태,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수
0,78,35,20131204,계단식,120,51.89,19.2603,50758000,620370,109
1,78,43,20131204,계단식,120,59.93,22.2446,63166000,665490,109
2,35,26,20130801,복도식,47,27.75,16.5375,63062000,458640,35
3,35,9,20130801,복도식,47,29.08,17.3302,63062000,481560,35
4,88,7,20131023,계단식,106,59.47,21.9462,72190000,586540,88
...,...,...,...,...,...,...,...,...,...,...
1152,956,956,19941130,복도식,202,26.37,12.7500,9931000,134540,243
1153,120,66,20200110,복도식,40,24.83,15.1557,2129000,42350,47
1154,120,54,20200110,복도식,40,33.84,20.6553,2902000,57730,47
1155,447,149,19940615,복도식,89,26.37,13.3800,7134000,118880,78


# y_test는 결측치가 존재하지 않는 건물형태 y_test를 뽑아냄

In [148]:
y_test = apart.loc[apart['건물형태'].isnull()]
y_test = y_test['건물형태']

# 준공년도 전처리

In [151]:
apart['준공년도'] = apart['준공일자']//10000
apart.drop(columns='준공일자',inplace=True)
apart

,총세대수,전용면적별세대수,건물형태,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수,준공년도
0,78,35,계단식,120,51.89,19.2603,50758000,620370,109,2013
1,78,43,계단식,120,59.93,22.2446,63166000,665490,109,2013
2,35,26,복도식,47,27.75,16.5375,63062000,458640,35,2013
3,35,9,복도식,47,29.08,17.3302,63062000,481560,35,2013
4,88,7,계단식,106,59.47,21.9462,72190000,586540,88,2013
...,...,...,...,...,...,...,...,...,...,...
1152,956,956,복도식,202,26.37,12.7500,9931000,134540,243,1994
1153,120,66,복도식,40,24.83,15.1557,2129000,42350,47,2020
1154,120,54,복도식,40,33.84,20.6553,2902000,57730,47,2020
1155,447,149,복도식,89,26.37,13.3800,7134000,118880,78,1994


# x_test 추출

In [154]:
x_test = apart.loc[apart['건물형태'].isnull()] # 건물형태가 어떨 것이다 라고 예측해야할 데이터
x_test = x_test.drop(columns='건물형태')
x_cols = list(x_test)
x_test

,총세대수,전용면적별세대수,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수,준공년도
59,1228,165,1233,59.93,25.1097,44718000,483160,1421,2017
60,1228,1063,1233,59.98,25.1307,44718000,483160,1421,2017
196,947,200,1311,74.55,22.3216,63000000,360000,1384,2018
197,947,103,1311,74.65,22.3515,63000000,360000,1384,2018
198,947,105,1311,74.80,22.3964,63000000,360000,1384,2018
199,947,94,1311,84.74,25.3726,75000000,390000,1384,2018
200,947,290,1311,84.90,25.4205,75000000,390000,1384,2018
201,947,290,1311,84.99,25.4475,75000000,390000,1384,2018
202,526,112,727,74.55,22.3834,63000000,360000,862,2018
203,526,58,727,74.65,22.4134,63000000,360000,862,2018


# 학습할 x_train과 y_train을 추출하기 (결측치는 존재하지 않아야함) | x_train,x_test 정규화 작업

In [157]:
scaler = MinMaxScaler()

apart=apart.dropna(subset=['건물형태'])

target = '건물형태'
x_train = apart.drop(columns=target)
y_train = apart['건물형태']

x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train,columns=x_cols)

x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test,columns=x_cols)
x_test

,총세대수,전용면적별세대수,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수,준공년도
0,0.536276,0.130469,0.269205,0.347733,0.521802,0.175418,0.456660,0.855746,0.833333
1,0.536276,0.844869,0.269205,0.348144,0.522371,0.175418,0.456660,0.855746,0.833333
2,0.413462,0.158313,0.286375,0.467806,0.446264,0.247134,0.340255,0.833130,0.866667
3,0.413462,0.081146,0.286375,0.468627,0.447074,0.247134,0.340255,0.833130,0.866667
4,0.413462,0.082737,0.286375,0.469859,0.448290,0.247134,0.340255,0.833130,0.866667
5,0.413462,0.073986,0.286375,0.551495,0.528924,0.294208,0.368610,0.833130,0.866667
6,0.413462,0.229912,0.286375,0.552809,0.530222,0.294208,0.368610,0.833130,0.866667
7,0.413462,0.229912,0.286375,0.553548,0.530954,0.294208,0.368610,0.833130,0.866667
8,0.229458,0.088305,0.157825,0.467806,0.447938,0.247134,0.340255,0.514059,0.866667
9,0.229458,0.045346,0.157825,0.468627,0.448751,0.247134,0.340255,0.514059,0.866667


# 모델 선언 및 학습

In [160]:
model = KNeighborsClassifier()

In [161]:
model.fit(x_train,y_train)

KNeighborsClassifier()

In [164]:
model.score(x_train,y_train)

0.920704845814978

In [165]:
y_pred = model.predict(x_test) #예측하기 그리고 이 데이터를 사용할 것
y_pred

array(['계단식', '계단식', '계단식', '계단식', '계단식', '계단식', '계단식', '계단식', '계단식',
       '계단식', '계단식', '계단식', '계단식', '계단식', '복도식', '복도식', '복도식', '복도식',
       '복도식', '복도식', '복도식', '복도식'], dtype=object)

# 추출한 y_pred(건물형태 결측치)를 기존 데이터에 할당

In [169]:
# 데이터 다시 불러오기
apart = pd.read_excel(''+'train.xlsx')
apart

,단지코드,단지명,총세대수,전용면적별세대수,지역,준공일자,건물형태,난방방식,승강기설치여부,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수
0,C0001,엘에이치 서초4단지,78,35,서울,20131204,계단식,개별가스난방,전체동 설치,120,51.89,19.2603,50758000,620370,109
1,C0001,엘에이치 서초4단지,78,43,서울,20131204,계단식,개별가스난방,전체동 설치,120,59.93,22.2446,63166000,665490,109
2,C0002,LH삼성아파트,35,26,서울,20130801,복도식,개별가스난방,전체동 설치,47,27.75,16.5375,63062000,458640,35
3,C0002,LH삼성아파트,35,9,서울,20130801,복도식,개별가스난방,전체동 설치,47,29.08,17.3302,63062000,481560,35
4,C0003,강남LH8단지,88,7,서울,20131023,계단식,개별가스난방,전체동 설치,106,59.47,21.9462,72190000,586540,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,C0356,덕유마을 주공1단지,956,956,경기,19941130,복도식,지역가스난방,전체동 설치,202,26.37,12.7500,9931000,134540,243
1153,C0358,화천신읍(공공실버) 영구임대,120,66,강원,20200110,복도식,NaN,전체동 설치,40,24.83,15.1557,2129000,42350,47
1154,C0358,화천신읍(공공실버) 영구임대,120,54,강원,20200110,복도식,NaN,전체동 설치,40,33.84,20.6553,2902000,57730,47
1155,C0359,영천야사4,447,149,대구경북,19940615,복도식,중앙유류난방,전체동 설치,89,26.37,13.3800,7134000,118880,78


In [170]:
#기존 데이터 결측치 확인
apart.isna().sum()

단지코드         0
단지명          0
총세대수         0
전용면적별세대수     0
지역           0
준공일자         0
건물형태        22
난방방식        75
승강기설치여부     98
단지내주차면수      0
전용면적         0
공용면적         0
임대보증금        0
임대료          0
실차량수         0
dtype: int64

In [173]:
# 건물형태 컬럼의 결측치 인덱스를 찾음
missing_indices = apart[apart['건물형태'].isnull()].index

# 결측치에 y_pred 값을 채움 (결측치 개수와 y_pred 길이가 같아야 함)
apart.loc[missing_indices, '건물형태'] = y_pred

In [175]:
# 다시 결측치 확인
apart.isna().sum()

단지코드         0
단지명          0
총세대수         0
전용면적별세대수     0
지역           0
준공일자         0
건물형태         0
난방방식        75
승강기설치여부     98
단지내주차면수      0
전용면적         0
공용면적         0
임대보증금        0
임대료          0
실차량수         0
dtype: int64

# 결측치 채운 건물형태를 새로운 데이터 생성

In [180]:
# 데이터프레임을 joblib을 이용해 저장
joblib.dump(apart, 'new_data1.pkl')

['new_data1.pkl']

y_test = pd.DataFrame(y_test)
y_test['건물형태'] = y_pred

joblib.dump(y_test, ''+'건물형태_예측.pkl')